In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn import model_selection

from matplotlib import pyplot as plt
from matplotlib import cm

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../')
from ml2grb.sklearn2grb import Pipe2Gurobi

In [ ]:
x = np.arange(-1, 1, 0.005)

z = (x)**2


In [ ]:
plt.plot(x, z)

plt.show()

In [ ]:
X = np.concatenate([x.ravel().reshape(-1, 1)], axis=1)
Y = z.ravel()

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,Y)

In [ ]:
# Run our regression
regression =  DecisionTreeRegressor(max_depth=10)
regression.fit(X=X_train, y=y_train)

In [ ]:
plt.plot(X, regression.predict(X))

plt.show()

In [ ]:
node = 0
tree = regression.tree_
x = X[200,:]
while tree.children_left[node] > 0:
    if x[tree.feature[node]] <= tree.threshold[node]:
        node = tree.children_left[node]
    else:
        node = tree.children_right[node]
    

In [ ]:
regression.predict(X[200:201,:])

### Do the optimization model

In [ ]:
npoints = 1
optfeat = [0]

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((npoints, len(optfeat)), lb=-1, ub=1, name='x')
y = m.addMVar((npoints,1), lb=-GRB.INFINITY, name='y')

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

In [ ]:
tree = regression.tree_

In [ ]:
nodes = m.addMVar(tree.capacity, vtype=GRB.BINARY)

In [ ]:
root = nodes[0]

In [ ]:
m.addConstrs( nodes[i] >= nodes[j] for i, j in enumerate(tree.children_left) if j >= 0)
m.addConstrs( nodes[i] >= nodes[j] for i, j in enumerate(tree.children_right) if j >= 0)
m.addConstrs( nodes[i] <= nodes[j] + nodes[k] for i, (j, k) in enumerate(zip(tree.children_right, tree.children_left)) if j >= 0)
m.addConstrs( 1 >= nodes[j] + nodes[k] for j, k in zip(tree.children_right, tree.children_left) if j >= 0)

In [ ]:
for node in range(tree.capacity):
    left = tree.children_left[node]
    right =tree.children_right[node]
    if left < 0:
        m.addConstr((nodes[node] == 1) >> (y[0,0] == tree.value[node][0][0]))
        continue
    m.addConstr((nodes[left] == 1) >> (x[0,tree.feature[node]] <= tree.threshold[node]))
    m.addConstr((nodes[right] == 1) >> (x[0,tree.feature[node]] >= tree.threshold[node] + 0.01))
    # m.addConstr(x[0,tree.feature[node]] <= tree.threshold[node] + 2 * (1 - nodes[left]))
    # m.addConstr(x[0,tree.feature[node]] >= tree.threshold[node] + 0.01 - 2 * (1 - nodes[right]))
    
    

In [ ]:
m.addConstr( gp.quicksum([nodes[i]
                           for i in range(tree.capacity)
                           if tree.children_left[i] < 0]) == 1)

In [ ]:
y.LB = np.min(tree.value)
y.UB = np.max(tree.value)

In [ ]:
from gurobipy import tupledict

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 200
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

In [ ]:
x.X

### Look at the solution

In [ ]:
x.X

In [ ]:
y.X

In [ ]:
regression.predict(x.X)

In [ ]:
tree.value

Copyright © 2020 Gurobi Optimization, LLC